In [1]:
######### loading the packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
import ipywidgets as widgets
from IPython.display import display
import voila

In [2]:
# Instead of a global directory, just use relative folders:
data_directory = "data"
graph_directory = "graph"

# Ensure these folders exist (Binder will create them fresh each session)
# os.makedirs(data_directory, exist_ok=True)
# os.makedirs(graph_directory, exist_ok=True)

In [32]:
# df_rel = pd.read_csv(os.path.join(data_directory, 'GpbaRel_vest.csv'))
# # for col in df_abs.columns:
#     # print(col)
# df_rel['value'] = df_rel[['nonEquityTarget', 'grantDateFV']].bfill(axis=1).iloc[:, 0]
# columns_to_keep = ['fiscalYear', 'grantId', 'metric', 'size', 'CIK', 'participantid', 'grantDate', 'startDate', 'endDate', 'companyName']
# df_rel['size'] = df_rel['value']*df_rel['percentVest']
# df_rel = df_rel[columns_to_keep]
# df_rel.to_csv(os.path.join(data_directory, 'GpbaRel_vest_clean.csv'), index=False)
# # df_rel.to_csv(os.path.join(data_directory, 'GpbaRel_vest.csv'), index=False)

In [33]:
# df_abs = pd.read_csv(os.path.join(data_directory, 'GpbaAbs_vest.csv'))
# # for col in df_abs.columns:
#     # print(col)
# df_abs['value'] = df_abs[['nonEquityTarget', 'grantDateFV']].bfill(axis=1).iloc[:, 0]
# columns_to_keep = ['fiscalYear', 'grantId', 'metric', 'size', 'CIK', 'participantid', 'grantDate', 'startDate', 'endDate', 'companyName']
# df_abs['size'] = df_abs['value']*df_abs['percentVest']
# df_abs = df_abs[columns_to_keep]
# df_abs.to_csv(os.path.join(data_directory, 'GpbaAbs_vest_clean.csv'), index=False)
# # df_abs.to_csv(os.path.join(data_directory, 'GpbaAbs_vest.csv'), index=False)

In [7]:
base = pd.read_csv(os.path.join(data_directory, 'SumComp.csv'))
base['CIK'] = base['CIK'].astype(str).str.replace(r"[^\d]", "", regex=True).astype(int)
# # base.to_csv(os.path.join(data_directory, 'SumComp.csv'), index=False)
# print(base.head)

In [4]:
df_comp = pd.read_csv(os.path.join(data_directory, 'GpbaAbs_vest_clean.csv'), encoding="latin1")
df_comp['CIK'] = df_comp['CIK'].astype(str).str.replace(r"[^\d]", "", regex=True).astype(int)
# # # 1) Check for multiple companyNames per CIK
# # cik_name_counts = df_comp.groupby('CIK')['companyName'].nunique()
# # ciks_multiple_names = cik_name_counts[cik_name_counts > 1].index

# # # if len(ciks_multiple_names) > 0:
# # #     print("CIKs that map to multiple companyName values:")
# # #     for bad_cik in ciks_multiple_names:
# # #         # Print all rows for this CIK, showing the different companyNames
# # #         subset = df_comp[df_comp['CIK'] == bad_cik][['CIK', 'companyName']].drop_duplicates()
# # #         print(subset)
# # # else:
# # #     print("No CIK maps to multiple companyName values.")

# # # print("----")

# # # 2) Check for multiple CIKs per companyName
# # name_cik_counts = df_comp.groupby('companyName')['CIK'].nunique()
# # names_multiple_ciks = name_cik_counts[name_cik_counts > 1].index

# # if len(names_multiple_ciks) > 0:
# #     print("companyName values that map to multiple CIKs:")
# #     for bad_name in names_multiple_ciks:
# #         # Print all rows for this companyName, showing the different CIKs
# #         subset = df_comp[df_comp['companyName'] == bad_name][['CIK', 'companyName']].drop_duplicates()
# #         print(subset)
# # else:
# #     print("No companyName maps to multiple CIK values.")

company_list = (
    df_comp[['CIK', 'companyName']]
    .drop_duplicates()
    .values
    .tolist()
)

In [19]:
df_ceo = pd.read_csv(os.path.join(data_directory, 'GpbaAbs_vest_clean.csv'), encoding="latin1",low_memory=False)
df_ceo['CIK'] = df_ceo['CIK'].astype(str).str.replace(r"[^\d]", "", regex=True).astype(int)
df_ceo['participantid'] = df_ceo['participantid'].astype(int)
# # print(df_ceo)
ceo_list = (
    df_ceo[['CIK', 'participantid']]
    .drop_duplicates()
    .values
    .tolist()
)
# print(ceo_list)

In [33]:
def create_df_combined(company_name=None, CIK=None, participantid=None, dis=False):
    """
    Reads the CSV files, filters on company_name (substring match, case-insensitive),
    CIK, and participantid if they are provided. Returns the combined DataFrame with
    columns: [CIK, companyName, metric, fiscalYear, grantDate, startDate, endDate, value, absRel].
    """

    # --- Read CSVs ---
    df_abs = pd.read_csv(os.path.join(data_directory, 'GpbaAbs_vest.csv'))
    df_rel = pd.read_csv(os.path.join(data_directory, 'GpbaRel_vest.csv'))

    # --- Filter the DataFrames ---
    if company_name:
        df_abs_filtered = df_abs[(df_abs['companyName'].str.contains(company_name, case=False, na=False))& (df_abs['participantid'] == float(participantid))]
        df_rel_filtered = df_rel[(df_rel['companyName'].str.contains(company_name, case=False, na=False))& (df_rel['participantid'] == float(participantid))]
    else:
        df_abs_filtered = df_abs[(df_abs['CIK'] == CIK) & (df_abs['participantid'] == float(participantid))]
        df_rel_filtered = df_rel[(df_rel['CIK'] == CIK) & (df_rel['participantid'] == float(participantid))]

    if df_abs_filtered.empty and df_rel_filtered.empty:
        print(f"No data found for CIK={CIK}, participant={participantid}.")
        return  # nothing else to plot
    
    # --- Select columns and label abs/rel ---
    columns_to_display = [
        'CIK', 'companyName', 'metric', 'fiscalYear', 
        'grantDate', 'startDate', 'endDate', 'size', 'percentVest', 'value', 'grantId'
    ]
    df_abs_filtered = df_abs_filtered[columns_to_display].copy()
    df_rel_filtered = df_rel_filtered[columns_to_display].copy()

    df_abs_filtered['absRel'] = 'abs'
    df_rel_filtered['absRel'] = 'rel'

    df_combined = pd.concat([df_abs_filtered, df_rel_filtered], ignore_index=True)
    df_combined = df_combined.sort_values(by=['metric', 'grantDate'], ascending=[True, True])

    # Display the combined DataFrame (in notebook)
    if dis:
        display(df_combined)

    return df_combined

In [9]:
def plot_compensation_timeline(company_name, CIK, participantid, dis=False):
    """
    Reads the vest CSV files, filters on CIK & participantid,
    displays the combined DataFrame, and plots the timeline.
    """
    df_combined = create_df_combined(company_name=company_name, CIK=CIK, participantid=participantid, dis=dis)
    if df_combined.empty:
        print(f"No data found for company={company_name}, CIK={CIK}, participant={participantid}.")
        return

    # --- Convert date columns to datetime ---
    df_combined['grantDate'] = pd.to_datetime(df_combined['grantDate'], errors='coerce')
    df_combined['startDate'] = pd.to_datetime(df_combined['startDate'], errors='coerce')
    df_combined['endDate']   = pd.to_datetime(df_combined['endDate'], errors='coerce')

    # --- Retrieve base salary data ---
    if pd.isna(CIK):  # If company_name is used, map it to CIK
        CIK = df_combined['CIK'].iloc[0] if not df_combined.empty else None

    base_filtered = base[(base['CIK'] == CIK) & (base['participantid'] == participantid)]

    # Convert fiscalYear to integer for sorting
    base_filtered.loc[:,'FiscalYear'] = pd.to_numeric(base_filtered['FiscalYear'], errors='coerce')

    # --- Compute the minimum value for scaling line thickness ---
    min_val = pd.concat([df_combined['size'], base_filtered['salary']], axis=0).dropna().min()

    # --- Sort metrics by frequency ---
    metric_counts = df_combined['metric'].value_counts(ascending=True)
    sorted_metrics = metric_counts.index.tolist()

    df_combined['metric'] = pd.Categorical(
        df_combined['metric'], 
        categories=sorted_metrics, 
        ordered=True
    )
    df_combined = df_combined.sort_values(by=['metric', 'grantDate'])

    # --- Set up color map, positions, etc. ---
    palette = sns.color_palette("colorblind", 10)
    color_map = {'abs': palette[0], 'rel': palette[1]}

    metric_positions = {}
    y_positions = []
    metric_labels = {}

    current_y = 2
    for metric in sorted_metrics:
        metric_rows = df_combined[df_combined['metric'] == metric]
        metric_positions[metric] = current_y
        row_count = len(metric_rows)
        for _ in range(row_count):
            y_positions.append(current_y)
            current_y += 1
        # Center label between these bars
        metric_labels[metric] = current_y - (row_count / 2)
        current_y += 1  # blank row between metrics

    df_combined['y_pos'] = y_positions

    # --- Plot ---
    fig, ax = plt.subplots(figsize=(12, 6))

    legend_labels = {
        'absolute, performance period': False,
        'relative, performance period': False,
        'Grant Date': False
    }

    min_val = df_combined['size'].dropna().min()
    for _, row in df_combined.iterrows():
        y = row['y_pos']
        if row['absRel'] == 'abs':
            line_label = 'absolute, performance period'
        else:
            line_label = 'relative, performance period'

        # Decide line style and thickness
        if pd.isna(row['size']):
            ls = '--'           # dashed line if value is NaN
            lw = 1             # pick a default thickness for NaN rows
        else:
            ls = '-'           # solid line if value is not NaN
            # example scaling: if 'value' is large, you can divide or clamp it
            # e.g. ensure minimum thickness of 1
            lw = 1 * row['size'] / min_val
            # adjust divisor or logic to suit your data distribution
    
        # performance period bar
        color = color_map[row['absRel']]
        if not legend_labels[line_label]:
            ax.plot([row['startDate'], row['endDate']], [y, y], 
                    color=color, linewidth=lw, linestyle=ls,
                    label=line_label)
            legend_labels[line_label] = True
        else:
            ax.plot([row['startDate'], row['endDate']], [y, y],
                    color=color, linewidth=lw, linestyle=ls)

        # grant date marker
        if not legend_labels['Grant Date']:
            ax.scatter(row['grantDate'], y, color='black', marker='o', label='Grant Date')
            legend_labels['Grant Date'] = True
        else:
            ax.scatter(row['grantDate'], y, color='black', marker='o')

    # --- Plot Base Salary ---
    base_salary_y = 0  # Position below the lowest metric
    base_salary_legend_added = False
    for _, row in base_filtered.iterrows():
        fiscal_year = int(row['FiscalYear'])
        salary_value = row['salary']
        if pd.notna(salary_value):
            lw = 1 * salary_value / min_val  # Scale thickness
            ax.plot([pd.Timestamp(f"{fiscal_year}-01-01"), pd.Timestamp(f"{fiscal_year}-12-31")], 
                    [base_salary_y, base_salary_y], color='black', linewidth=lw, label='Base Salary')
            base_salary_legend_added = True 
            
    # horizontal dashed lines to separate metrics
    for metric, pos in metric_positions.items():
        ax.axhline(y=pos - 1, color='gray', linestyle='dashed', alpha=0.3)

    # Label y-axis for base salary
    # ax.text(pd.Timestamp(f"{base_filtered['FiscalYear'].min()}-01-01"), base_salary_y, 
            # "Base Salary", ha='right', va='center', fontsize=10, color='black')

    ax.set_xlabel("Time")

    # Title (use the first non-null companyName)
    if df_combined['companyName'].dropna().empty:
        title = f"Metric-Level Timeline (CIK={CIK}, participant={participantid})"
    else:
        title = f"Metric-Level Timeline for {df_combined['companyName'].iloc[0]}, CEO {participantid}"
    ax.set_title(title)

    # y-ticks
    ax.set_yticks(list(metric_labels.values()) + [base_salary_y])
    ax.set_yticklabels(list(metric_labels.keys()) + ["Base Salary"])

    # Format x-axis as Year-Month
    ax.xaxis.set_major_locator(mdates.YearLocator())
    ax.xaxis.set_minor_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    plt.xticks(rotation=45)

    # Legend ordering
    handles, labels = ax.get_legend_handles_labels()
    desired_order = ['absolute, performance period', 'relative, performance period', 'Grant Date', 'Base Salary']
    ordered_handles = [handles[labels.index(lbl)] for lbl in desired_order if lbl in labels]
    plt.legend(ordered_handles, desired_order, loc='upper left')

    plt.grid(axis='x', linestyle='--', alpha=0.6)
    plt.tight_layout()

    # Optionally save figure
    # This line saves it under timeline/companyName.jpg if you want
    # If there's a valid companyName, otherwise fallback:
    safe_name = title.replace(" ", "_").replace("/", "_")
    plot_path = os.path.join(graph_directory, f"{safe_name}_metric_timeline.jpg")
    plt.savefig(plot_path)

    # print(f"Plot saved to: {plot_path}")
    plt.show()

In [10]:
def plot_compensation_timeline_by_grant(company_name, CIK, participantid, dis=False):
    """
    Group df_combined by 'grantId'. 
    For each group, determine distinct (grantDate, startDate, endDate) combos.
    Plot one horizontal line per distinct combo. 
    Label the y-axis with:
      - if there's only one distinct timeline, show the list of metrics in the group
      - if multiple timelines exist, show separate lines, labeling each line with the relevant metrics
    """
    df_combined = create_df_combined(company_name=company_name, CIK=CIK, participantid=participantid, dis=dis)
    if df_combined.empty:
        print(f"No data found for company={company_name}, CIK={CIK}, participant={participantid}.")
        return

    # Just in case, ensure these are datetimes
    df_combined['grantDate'] = pd.to_datetime(df_combined['grantDate'], errors='coerce')
    df_combined['startDate'] = pd.to_datetime(df_combined['startDate'], errors='coerce')
    df_combined['endDate']   = pd.to_datetime(df_combined['endDate'], errors='coerce')

    # --- Retrieve base salary data ---
    if pd.isna(CIK):  # If company_name is used, map it to CIK
        CIK = df_combined['CIK'].iloc[0] if not df_combined.empty else None

    base_filtered = base[(base['CIK'] == CIK) & (base['participantid'] == participantid)]

    # Convert fiscalYear to integer for sorting
    base_filtered.loc[:,'FiscalYear'] = pd.to_numeric(base_filtered['FiscalYear'], errors='coerce')
    
    # --- Compute the minimum value for scaling line thickness ---
    min_value = pd.concat([df_combined['size'], base_filtered['salary']], axis=0).dropna().min()

    # Group by grantId
    grouped = df_combined.groupby('grantId', dropna=False)
    group_grant_dates = {grantId: sub_df['grantDate'].min() for grantId, sub_df in grouped}
    sorted_groups = sorted(group_grant_dates.items(), key=lambda x: x[1])

    # Prepare the figure
    fig, ax = plt.subplots(figsize=(12, 6))

    # For coloring, reuse your color scheme if you want
    palette = sns.color_palette("colorblind", 10)
    color_map = {'abs': palette[0], 'rel': palette[1]}
    
    # We'll keep track of the y-position for each group
    current_y = 2
    y_positions = []
    y_labels = []

    # Track legend entries
    legend_labels = {
        'absolute, performance period': False,
        'relative, performance period': False,
        'Grant Date': False,
        'Base Salary': False
    }
    sorted_grant_ids = sorted(grouped.groups.keys(), key=lambda x: group_grant_dates[x])

    for grant_id in sorted_grant_ids:
        ax.axhline(y=current_y, color='gray', linestyle='dashed', alpha=0.3)
        group_df = grouped.get_group(grant_id)  # Retrieve the actual DataFrame
        label_for_group = f"GrantID={grant_id}" if pd.notna(grant_id) else "Unknown GrantId"

        # Identify distinct combos within this group
        combo_groups = group_df.groupby(['grantDate', 'startDate', 'endDate'], dropna=False)
        
        # If only 1 distinct timeline, we label the y-axis with all metrics from the entire group
        # else, we label each timeline line with its metrics
        if len(combo_groups) == 1:
            # There's just one combo => we can directly get that
            (gDate, sDate, eDate), sub_df = list(combo_groups)[0]
            
            # Plot one line
            first_value = sub_df['value'].iloc[0] if not sub_df['value'].isna().all() else min_value
            total_height = 2 * (first_value / min_value) if min_value else 2
            # 1) Check if percentVest has NaNs
            if sub_df['percentVest'].isna().any():
                use_count_based = True
                total_count = len(sub_df)
                sub_df['computedVest'] = total_height / total_count  # Equal height for all rows
                linestyle = '--'  # Dashed lines since we are using fallback
            else:
                use_count_based = False
                total_percentVest = sub_df['percentVest'].sum()
                if total_percentVest == 0 or pd.isna(total_percentVest):
                    total_percentVest = 1  # Avoid division by zero
                sub_df['computedVest'] = sub_df['percentVest'] / total_percentVest * total_height
                linestyle = '-'  # Solid lines for percent-based stacking
        
            # 2) Sort by 'absRel' (absolute first, then relative), then by height (largest to smallest)
            sub_df = sub_df.sort_values(by=['absRel', 'computedVest'], ascending=[False, True])
            sorted_metrics = sub_df['metric'].tolist()[::-1]  # Get metric names in the same order as stacks
            metric_label = ", ".join(sorted_metrics) if sorted_metrics else "No Metric"

            # 3) Compute the vertical stacking based on computedVest
            start_y = current_y
            for _, row in sub_df.iterrows():
                height = row['computedVest']
                color = color_map.get(row['absRel'], 'gray')

                line_label = 'absolute, performance period' if row['absRel'] == 'abs' else 'relative, performance period'

                # Draw stacked vertical section
                if not legend_labels[line_label]:
                    ax.plot([sDate, eDate],
                            [start_y + height/2, start_y + height/2],
                            color=color, linestyle=linestyle, linewidth=height, label=line_label)
                    legend_labels[line_label] = True
                else:
                    ax.plot([sDate, eDate],
                            [start_y + height/2, start_y + height/2],
                            color=color, linestyle=linestyle, linewidth=height)
        
                # Move up to stack the next segment
                start_y += height
            
            # Plot the grant date marker
            ax.scatter(gDate, current_y + total_height / 2, color='black', marker='o', label="Grant Date" if not legend_labels['Grant Date'] else "", zorder=5)
            legend_labels['Grant Date'] = True

            # We'll label the entire group's y-tick with the list of metrics
            y_positions.append(current_y+ total_height/2)
            y_labels.append(f"{label_for_group}\n({metric_label})")

            current_y += total_height + 1  # move down for next group

        else:
            # Multiple distinct combos => each timeline gets its own line
            # We'll label each line with the relevant metrics
            base_y = current_y
            lines_in_group = 0

            for (gDate, sDate, eDate), sub_df in combo_groups:
                # gather metrics for *this timeline*
                first_value = sub_df['value'].iloc[0] if not sub_df['value'].isna().all() else min_value
                total_height = 1 * (first_value / min_value) if min_value else 1
                # 1) Check if percentVest has NaNs
                if sub_df['percentVest'].isna().any():
                    use_count_based = True
                    total_count = len(sub_df)
                    sub_df['computedVest'] = total_height / total_count  # Equal height for all rows
                    linestyle = '--'  # Dashed lines since we are using fallback
                else:
                    use_count_based = False
                    total_percentVest = sub_df['percentVest'].sum()
                    if total_percentVest == 0 or pd.isna(total_percentVest):
                        total_percentVest = 1  # Avoid division by zero
                    sub_df['computedVest'] = sub_df['percentVest'] / total_percentVest * total_height
                    linestyle = '-'  # Solid lines for percent-based stacking

                # 2) Sort by 'absRel' (absolute first, then relative), then by height (largest to smallest)
                sub_df = sub_df.sort_values(by=['absRel', 'computedVest'], ascending=[False, True])
                sorted_metrics = sub_df['metric'].tolist()[::-1]  # Get metric names in the same order as stacks
                metric_label = ", ".join(sorted_metrics) if sorted_metrics else "No Metric"
            
                # 3) Compute the vertical stacking based on computedVest
                start_y = current_y
                for _, row in sub_df.iterrows():
                    height = row['computedVest']
                    color = color_map.get(row['absRel'], 'gray')
            
                    line_label = 'absolute, performance period' if row['absRel'] == 'abs' else 'relative, performance period'

                    # Draw stacked vertical section
                    if not legend_labels[line_label]:
                        ax.plot([sDate, eDate],
                            [start_y + height/2, start_y + height/2],
                            color=color, linestyle=linestyle, linewidth=height)
                        legend_labels[line_label] = True
                    else:
                        ax.plot([sDate, eDate],
                            [start_y + height/2, start_y + height/2],
                            color=color, linestyle=linestyle, linewidth=height)
                    # Move up to stack the next segment
                    start_y += height

                # Add a text annotation near the middle of the line with the metrics
                mid_x = sDate + (eDate - sDate) / 2 if (pd.notna(sDate) and pd.notna(eDate)) else sDate
                ax.text(mid_x, current_y + total_height / 2, metric_label, ha='center', va='bottom', fontsize=9, color='black')

                # Plot the grant date marker
                ax.scatter(gDate, current_y + total_height / 2, color='black', marker='o', label="Grant Date" if not legend_labels['Grant Date'] else "", zorder=5)
                legend_labels['Grant Date'] = True

                current_y += total_height + 1
                lines_in_group += 1

            # after multiple lines, we can label the base_y with the group name
            # e.g. put a label at the midpoint of those lines
            group_center_y = base_y + (lines_in_group - 1) / 2
            y_positions.append(group_center_y)
            y_labels.append(label_for_group)
            current_y += 1  # blank space between groups

    # --- Plot Base Salary ---
    base_salary_y = 0  # Position below the lowest metric
    for _, row in base_filtered.iterrows():
        fiscal_year = int(row['FiscalYear'])
        salary_value = row['salary']
        if pd.notna(salary_value):
            lw = 2 * salary_value / min_value  # Scale thickness
            ax.plot([pd.Timestamp(f"{fiscal_year}-01-01"), pd.Timestamp(f"{fiscal_year}-12-31")], 
                    [base_salary_y, base_salary_y], color='black', linewidth=lw, label='Base Salary')
            legend_labels['Base Salary'] = True

    # Decorate axes
    ax.set_xlabel("Time")
    ax.set_ylabel("Grant-level grouping")

    # Format x-axis as Year-Month
    ax.xaxis.set_major_locator(mdates.YearLocator())
    ax.xaxis.set_minor_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    plt.xticks(rotation=45)

    # ax.set_yticks(y_positions)
    # ax.set_yticklabels(y_labels)
    # y-ticks
    ax.set_yticks(y_positions + [base_salary_y])
    ax.set_yticklabels(y_labels + ["Base Salary"])

    ax.grid(axis='x', linestyle='--', alpha=0.6)

    # --- Ensure Proper Legend ---
    handles, labels = ax.get_legend_handles_labels()
    desired_order = ['absolute, performance period', 'relative, performance period', 'Grant Date', 'Base Salary']
    final_labels = [lbl for lbl in desired_order if legend_labels.get(lbl, False)]
    ordered_handles = [handles[labels.index(lbl)] for lbl in final_labels if lbl in labels]
    ax.legend(ordered_handles, final_labels, loc='upper left')

    plt.tight_layout()
    plt.title(f"Grant-Level Timeline for {df_combined['companyName'].iloc[0]}, CEO {participantid}")
    plt.show()

In [11]:
# ######### Create input widgets
# company_widget = widgets.Text(value="Kellogg", description="Company:")
# CIK_widget = widgets.IntText(value=55067, description="CIK:")
# participant_widget = widgets.IntText(value=58579, description="CEO ID:")
# run_button = widgets.Button(description="Generate Timeline")

# # This output area will display the DataFrame and the plot
# output_area = widgets.Output()

# def on_run_button_click(b):
#     """
#     Clears the output area, then runs the plotting function.
#     """
#     with output_area:
#         output_area.clear_output()
#         company_val = company_widget.value
#         CIK_val = CIK_widget.value
#         participant_val = participant_widget.value
#         # plot_compensation_timeline(company_val, CIK_val, participant_val, False)
#         plot_compensation_timeline_by_grant(company_val, CIK_val, participant_val, False)
        
# run_button.on_click(on_run_button_click)

# # Display the widgets & output in the notebook
# widgets.VBox([
#     widgets.HBox([company_widget, CIK_widget, participant_widget]),
#     run_button,
#     output_area
# ])

In [36]:
# Build sets of all names, all CIKs:
all_company_names = sorted({nm for (_, nm) in company_list})
all_ciks = sorted({c for (c, _) in company_list})
all_ceos = sorted({p for (_, p) in ceo_list})

def filter_company_names(substring):
    """Return a list of company names containing 'substring' (case-insensitive)."""
    s_lower = substring.lower()
    return [n for n in all_company_names if s_lower in n.lower()]

def filter_ciks(substring):
    """Return a list of CIKs (as strings) containing 'substring'."""
    return [str(c) for c in all_ciks if substring in str(c)]

def filter_ceos(substring):
    """Return a list of CEOs (as strings) containing 'substring'."""
    return [str(p) for p in all_ceos if substring in str(p)]

def ciks_for_company(company_name):
    """All CIKs that match the given company_name in company_list, as strings."""
    return sorted({str(c) for (c, n) in company_list if n == company_name})

def names_for_cik(cik_str):
    """All company names that match the given CIK in company_list."""
    try:
        cik_int = int(cik_str)
        return sorted({n for (c, n) in company_list if c == cik_int})
    except ValueError:
        return []

def ceos_for_cik(cik_str):
    """All CEO names that match the given CIK in ceo_list."""
    try:
        cik_int = int(cik_str)
        return sorted({str(item[1]) for item in ceo_list if int(item[0]) == cik_int})
    except ValueError:
        return []


# --- Create dynamic Comboboxes for Company & CIK ---
company_widget = widgets.Combobox(
    placeholder='Type a company name...',
    options=all_company_names,
    description='Company:'
)
# company_widget.value = "Kellogg"  # your existing default

CIK_widget = widgets.Combobox(
    placeholder='Type a CIK...',
    options=[str(c) for c in all_ciks],
    description='CIK:'
)
# CIK_widget.value = "55067"  # your existing default

participant_widget = widgets.Combobox(
    placeholder='Type a CEO ID...',
    options=[str(p) for p in all_ceos],
    description='CEO ID:'
)
# participant_widget.value = "58579"  # your existing default

run_button = widgets.Button(description="Generate Timeline")
output_area = widgets.Output()

# Observers for dynamic suggestions:
def on_company_typed(change):
    if change['name'] == 'value':
        typed = change['new'].strip()  # Remove extra whitespace
        typed = change['new']
        # Filter company suggestions
        company_widget.options = [
            n for n in all_company_names 
            if typed.lower() in n.lower()
        ]
        
        # If typed is an exact known name => narrow CIK options
        if typed in all_company_names:
            # Show only the matched CIK(s)
            matched_ciks = ciks_for_company(typed)
            CIK_widget.options = matched_ciks

            # If the current typed CIK isn't in matched_ciks, clear the CIK
            if CIK_widget.value not in matched_ciks:
                CIK_widget.value = ""  # Clear if mismatch
        else:
            # Not a finalized name => you could restore full CIK list or partial filter
            CIK_widget.options = []
            CIK_widget.value = ""
            pass

def on_cik_typed(change):
    if change['name'] == 'value':
        typed = change['new'].strip()
        print("CIK typed:", typed)  # Debug print
        # Filter suggestions for CIK
        CIK_widget.options = filter_ciks(typed)
        known_ciks = [str(c) for c in all_ciks]
        # print("Known CIKs:", known_ciks)  # Debug print
        
        # If typed is exactly one known CIK:
        if typed in known_ciks:
            # Show only the matched company name(s)
            matched_names = names_for_cik(typed)
            company_widget.options = matched_names
            
            matched_ceos = ceos_for_cik(typed)
            print("Matched CEOs for CIK", typed, ":", matched_ceos)  # Debug print
            participant_widget.options = matched_ceos

            # If the current typed company isn't in matched_names, clear the company
            if company_widget.value not in matched_names:
                company_widget.value = ""
            if participant_widget.value not in matched_ceos:
                participant_widget.value = ""
        else:
            # Not a finalized CIK => you could restore full name list or partial filter
            company_widget.options = []
            company_widget.value = ""
            participant_widget.options = filter_ceos(typed)
            participant_widget.value = ""

company_widget.observe(on_company_typed, names='value')
CIK_widget.observe(on_cik_typed, names='value')


def on_run_button_click(b):
    """
    Clears the output area, then runs the plotting function.
    """
    with output_area:
        output_area.clear_output()
        
        company_val = company_widget.value
        cik_val_str = CIK_widget.value
        try:
            CIK_val = int(cik_val_str)  # convert typed string to int
        except ValueError:
            CIK_val = None  # or handle error
        participant_val = participant_widget.value

        # Now call your timeline function(s)
        plot_compensation_timeline(company_val, CIK_val, participant_val, False)
        plot_compensation_timeline_by_grant(company_val, CIK_val, participant_val, False)

run_button.on_click(on_run_button_click)

# Display the widgets & output in the notebook
display(
    widgets.VBox([
        widgets.HBox([company_widget, CIK_widget, participant_widget]),
        run_button,
        output_area
    ])
)
